In [1]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_id = train_df["PassengerId"].values
test_id = test_df["PassengerId"].values

all_df = train_df.append(test_df).set_index('PassengerId')
all_df["Sex"] = all_df["Sex"].replace({"male":0,"female":1})

all_df["Age"].fillna(
    all_df.groupby("Pclass")["Age"].transform("mean"), inplace=True)
all_df["cabin_count"] = all_df["Cabin"].map(
         lambda x : len(x.split()) if type(x) == str else 0)
def transform_status(x):
    if "Mrs" in x or "Ms" in x:
        return "Mrs"
    elif "Mr" in x:
        return "Mr"
    elif "Miss" in x:
        return "Miss"
    elif "Master" in x:
        return "Master"
    elif "Dr" in x:
        return "Dr"
    elif "Rev" in x:
        return "Rev"
    elif "Col" in x:
        return "Col"
    else:
        return "0"

all_df["social_status"] = all_df["Name"].map(lambda x : transform_status(x))
all_df["social_status"].value_counts()
#all_df[all_df["Embarked"].isnull()]
all_df = all_df.drop([62,830])
train_id =np.delete(train_id, [62-1,830-1])
#all_df[all_df["Fare"].isnull()]
all_df.groupby(["Pclass","Sex"])["Fare"].mean()
all_df.loc[all_df["Fare"].isnull(), "Fare"] = 12.415462
all_df["cabin_type"] = all_df["Cabin"].map(lambda x : x[0] if type(x) == str else "99")
del all_df["Cabin"]
del all_df["Name"]
del all_df["Ticket"]
y = all_df.loc[train_id, "Survived"].values
del all_df["Survived"]
X_df = pd.get_dummies(all_df)
X = X_df.values
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
X = mm.fit_transform(X)

In [3]:
y.shape,X.shape

((889,), (1307, 27))

In [4]:
t_x = X[:len(train_id)]
tt_x = X[len(train_id):]

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
t_l = []
tt_l = []
for i in range(3,20):
    dt = DecisionTreeClassifier(min_samples_leaf=i)
    acc = cross_val_score(dt, t_x, y, scoring='accuracy', cv=5).mean()
    t_l.append(accuracy_score(dt.fit(t_x,y).predict(t_x),y))
    tt_l.append(acc)
r = pd.DataFrame(t_l, index=range(3,20),columns=['train'])
r['test']=tt_l
r.plot()